# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [3]:
# !pip install selenium - Biblioteca de automação de browser em segundo plano

In [3]:
import pandas as pd

# Bibliotecas de automação de navegador/browser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [29]:
# Criar o navegador
navegador = webdriver.Chrome()

# Entrar no google e pesquisar a cotação do dólar
navegador.get("https://www.google.com.br/")

# Pegar a cotação do dólar
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# Fechar o navegador após a consulta
navegador.quit()

print(cotacao_dolar)

5.0129


In [30]:
# Criar o navegador
navegador = webdriver.Chrome()

# Entrar no google e pesquisar a cotação do euro
navegador.get("https://www.google.com.br/")

# Pegar a cotação do euro
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# Fechar o navegador após a consulta
navegador.quit()

print(cotacao_euro)

5.53910869


In [31]:
# Criar o navegador
navegador = webdriver.Chrome()

# Entrar no google e pesquisar a cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

# Pegar a cotação do ouro
cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".") # Alterar a vírgula por ponto

# Fechar o navegador após a consulta
navegador.quit()

print(cotacao_ouro)

321.79


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [32]:
# Importar a base de dados
tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [33]:
# Atualização das cotações
# Cotação do dólar - tabela.loc(linha,coluna) = x
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar) # float indica que o valor será decimal
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# Atualização do Preço de Compra
tabela["Preço de Compra"] = tabela["Cotação"] * tabela["Preço Original"]

# Atualização do Preço de Venda
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.012900,5012.849871,1.40,7017.989819
1,Carro Renault,4500.00,Euro,5.539109,24925.989105,2.00,49851.978210
2,Notebook Dell,899.99,Dólar,5.012900,4511.559871,1.70,7669.651781
3,IPhone,799.00,Dólar,5.012900,4005.307100,1.70,6809.022070
4,Carro Fiat,3000.00,Euro,5.539109,16617.326070,1.90,31572.919533
5,Celular Xiaomi,480.48,Dólar,5.012900,2408.598192,2.00,4817.196384
6,Joia 20g,20.00,Ouro,321.790000,6435.800000,1.15,7401.170000


### Agora vamos exportar a nova base de preços atualizada

In [35]:
# Exportar a base de dados atualizada
tabela.to_excel("ProdutosNovo.xlsx", index = False) # Exporta sem índice